In [13]:
import torch
import coremltools as ct
import random

# Define a simple PyTorch model that uses grid sampling and conversion process


In [14]:
class PytorchGridSample(torch.nn.Module):
    def forward(self, input,grid):
        return torch.nn.functional.grid_sample(input,grid,align_corners = False)
    
def convert_to_coreml(model, inputs, is_float16=True):
    traced_model = torch.jit.trace(
        model, example_inputs=inputs, strict=False)

    coreml_model = ct.converters.convert(traced_model,
                                         inputs=[ct.TensorType(shape=inputs[0].shape),
                                                 ct.TensorType(shape=inputs[1].shape)],
                                         compute_precision=ct.precision.FLOAT16 if is_float16 else ct.precision.FLOAT32)
    return coreml_model


def compare_grid_samples_after_coreml_conversion(pt_model, inputs, is_float16):
    pt_out = pt_model(*inputs)
    coreml_pt_model = convert_to_coreml(pt_model, inputs, is_float16)
    input_names_coreml_pt = [i for i in coreml_pt_model.input_description]
    
    # Adjustment done foe the grid sample
    input_data = {
        input_names_coreml_pt[0]: inputs[0].detach().numpy(),
        input_names_coreml_pt[1]: inputs[1].permute(0, 2, 3, 1).contiguous().detach().numpy()
    }

    # Ensuring the grid shape to have correct shape of the tensor
    input_data[input_names_coreml_pt[1]] = input_data[input_names_coreml_pt[1]][:, :, :, [1, 0]]

    coreml_pt_out = torch.as_tensor(list(coreml_pt_model.predict(input_data).values())[0])
    diff_pt_coreml = torch.norm(coreml_pt_out - pt_out)
    return diff_pt_coreml
    

# Final state for the CoreML .pt values

In [15]:
# Output for the following
input_size = (1, 64, 2, 2)
grid_size = (1, 2, 2, 2)  

input_tensor = torch.randn(input_size).to(torch.float32)
grid = torch.randn(grid_size).to(torch.float32)
inputs = [input_tensor, grid]

pt_model = PytorchGridSample()

diff_pt_coreml_fp16 = compare_grid_samples_after_coreml_conversion(pt_model, inputs, is_float16=True)
diff_pt_coreml_fp32 = compare_grid_samples_after_coreml_conversion(pt_model, inputs, is_float16=False)

print(f"Difference between PyTorch's grid sample and CoreML converted model (FP16): {diff_pt_coreml_fp16}")
print(f"Difference between PyTorch's grid sample and CoreML converted model (FP32): {diff_pt_coreml_fp32}")
print(f"Relative change in the difference: {(diff_pt_coreml_fp16 - diff_pt_coreml_fp32) / diff_pt_coreml_fp32}")


When both 'convert_to' and 'minimum_deployment_target' not specified, 'convert_to' is set to "mlprogram" and 'minimum_deployment_targer' is set to ct.target.iOS15 (which is same as ct.target.macOS12). Note: the model will not run on systems older than iOS15/macOS12/watchOS8/tvOS15. In order to make your model run on older system, please set the 'minimum_deployment_target' to iOS14/iOS13. Details please see the link: https://coremltools.readme.io/docs/unified-conversion-api#target-conversion-formats
Model is not in eval mode. Consider calling '.eval()' on your model prior to conversion
Running MIL default pipeline:   0%|          | 0/71 [00:00<?, ? passes/s]/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/coremltools/converters/mil/mil/passes/defs/preprocess.py:267: UserWarning: Output, '6', of the source model, has been renamed to 'var_6' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Running MIL backend_mlprogram pipeline: 100%|███

Difference between PyTorch's grid sample and CoreML converted model (FP16): 8.556744575500488
Difference between PyTorch's grid sample and CoreML converted model (FP32): 8.556744575500488
Relative change in the difference: 0.0
